In [1]:
# Mount Google Drive on colab only
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# ─────────────────────────────────────────────────────────────
# 📁 Seed Titles
# ─────────────────────────────────────────────────────────────
SEED_TITLES = [
    # Programming & AI
    "Python_(programming_language)",
    "JavaScript",
    "Artificial_intelligence",
    "Machine_learning",
    "Natural_language_processing",
    "Transformer_(machine_learning_model)",
    "Git",
    "HTML",
    "CSS",
    "Neural_network",
    "Computer_vision",
    "Data_structure",
    "Algorithm",
    "Recursion_(computer_science)",
    "Big_O_notation",

    # Mythology & Lore
    "Sun_Wukong",
    "Journey_to_the_West",
    "Chinese_mythology",
    "Greek_mythology",
    "Norse_mythology",
    "Magic_(paranormal)",
    "Wand",
    "Alchemy",
    "Divination",
    "Horcrux",
    "Hogwarts",
    "Wizard",
    "Spell_(paranormal)",
    "Mythology",

    # Literature & Style
    "William_Shakespeare",
    "Shakespearean_language",
    "Poetry",
    "Sonnet",
    "Fantasy_literature",
    "Harry_Potter",
    "J._K._Rowling",
    "Literary_style",
    "Dialogue",
    "Narrative",
    "Character_(arts)",
    "Plot_(narrative)",
    "Symbolism",

    # Philosophy & Expression
    "Consciousness",
    "Creativity",
    "Emotion",
    "Language",
    "Semiotics",
    "Provenance",
    "Modularity",
    "Expressive_writing",
    "Storytelling"
]


In [ ]:
import requests
import json
import time
from pathlib import Path

# ─────────────────────────────────────────────────────────────
# 📁 Config
# ─────────────────────────────────────────────────────────────
MAX_ARTICLES = 100
#SAVE_DIR = Path("/content/drive/MyDrive/dataset/wiki_crawl") #google colab path
SAVE_DIR = Path("./dataset/wiki_crawl") #local path
SAVE_DIR.mkdir(parents=True, exist_ok=True)

HEADERS = {
    "User-Agent": "WEB-Crawler/1.0 (https://crazyg.is-a.dev or crazygiscool@proton.me)"
}

# ─────────────────────────────────────────────────────────────
# 🔧 Fetch Article Content via MediaWiki API
# ─────────────────────────────────────────────────────────────
def fetch_article(title):
    url = "https://en.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "prop": "extracts",
        "explaintext": True,
        "titles": title,
        "format": "json",
        "redirects": 1
    }
    response = requests.get(url, params=params, headers=HEADERS)
    if response.status_code != 200:
        raise ValueError(f"HTTP error {response.status_code}")
    try:
        data = response.json()
    except Exception as e:
        raise ValueError(f"Invalid JSON response: {e}")
    if "query" not in data or "pages" not in data["query"]:
        raise ValueError("Malformed response structure")
    page = next(iter(data["query"]["pages"].values()))
    return page.get("title"), page.get("extract")

# ─────────────────────────────────────────────────────────────
# 🧠 Format Content with Markdown and Glyphs
# ─────────────────────────────────────────────────────────────
def formatter(title, content):
    glyph = infer_glyph(title)
    lines = content.strip().split('\n')
    formatted = [f"<<glyph:{glyph}>>", f"# {title.replace('_', ' ')}", ""]

    for line in lines:
        line = line.strip()
        if not line:
            continue
        # Add section headers for long paragraphs
        if len(line.split()) > 12:
            formatted.append(f"## {line}")
        else:
            formatted.append(f"> {line}")

    return '\n'.join(formatted)

def infer_glyph(title):
    title = title.lower()
    if "python" in title or "html" in title or "css" in title or "javascript" in title:
        return "code"
    if "ai" in title or "machine" in title or "neural" in title or "transformer" in title:
        return "ai"
    if "sun wukong" in title or "journey" in title or "myth" in title:
        return "myth"
    if "shakespeare" in title or "poetry" in title or "sonnet" in title:
        return "literature"
    if "harry potter" in title or "wand" in title or "hogwarts" in title:
        return "magic"
    return "info"

# ─────────────────────────────────────────────────────────────
# 🔁 Safe Fetch with Retry
# ─────────────────────────────────────────────────────────────
def safe_fetch(title, retries=3, delay=2):
    for attempt in range(retries):
        try:
            return fetch_article(title)
        except Exception as e:
            print(f"⚠️ Attempt {attempt+1} failed for {title}: {e}")
            time.sleep(delay)
    return title, None

# ─────────────────────────────────────────────────────────────
# 🔁 Crawl from Seed Titles
# ─────────────────────────────────────────────────────────────
def crawl_wikipedia(seed_titles, max_articles):
    visited = set()
    queue = list(seed_titles)

    while queue and len(visited) < max_articles:
        title = queue.pop(0)
        if title in visited:
            continue

        page_title, content = safe_fetch(title)
        if content:
            formatted = formatter(page_title, content)
            filename = SAVE_DIR / f"{page_title.replace(' ', '_')}.md"
            with open(filename, "w", encoding="utf-8") as f:
                f.write(formatted)
            print(f"✅ Saved: {page_title}")
            visited.add(title)
        else:
            print(f"❌ Skipped: {title} (no content)")

        time.sleep(1.5)  # Respectful crawling

# ─────────────────────────────────────────────────────────────
# 🚀 Run Crawler
# ─────────────────────────────────────────────────────────────
crawl_wikipedia(SEED_TITLES, MAX_ARTICLES)
